<a href="https://colab.research.google.com/github/tothegump/Dash-User-Contributions/blob/master/myself_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import openai, os

openai.api_key = os.environ.get("OPENAI_API_KEY")

from langchain.prompts import PromptTemplate

# learn Agents

1. 输入源文章，判断语言
2. 划分段落，总结核心观点
3. 写新的 outline
4. 重要内容
5. 重新写文章

In [6]:
from pydantic import BaseModel, Field

# Define your desired data structure.
class Blogger(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked "
            "to extract, return null for the attribute's value.",
        ),
        # ↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
        MessagesPlaceholder("examples"),  # <-- EXAMPLES!
        # ↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑
        ("human", "{text}"),
    ]
)

# my agents


In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=ok)
from langchain_core.prompts import PromptTemplate



In [17]:
class IdeaGen(BaseModel):
    point: str = Field(description="the point around this topic")
    outline: str = Field(description="the outline about the point")


class IdeaList(BaseModel):
    idea_list: list[IdeaGen] = Field(description="the points and outline around this topic")

In [12]:
target = "a blog writer"
prompt_generator = PromptTemplate.from_template("""
As a prompt expert, you pay close attention to several characteristics of a prompt:
Clarity and Definiteness
Clear Background Context
Clear Steps or Instructions
Presence of Limiting Conditions
Use of Separators to Distinguish Independent Sections
Encouraging the Model to Check if Conditions are Met
===
now make a prompt about `{target}`, if you have something not clear, do not answer, just ask for making the target clear
""")
res = prompt_generator | llm

In [20]:
r = res.invoke({"target": target})

In [11]:
r

AIMessage(content="1. Share your top tips for balancing work and blogging.\n2. Discuss the importance of consistency in blog content.\n3. Write a review of your favorite blogging tools and resources.\n4. Share your experience with building a successful blog from scratch.\n5. Discuss the benefits of collaborating with other bloggers or brands.\n6. Write about the impact of social media on your blog's growth.\n7. Share your favorite blogging mistakes and what you learned from them.\n8. Discuss the importance of SEO and how it can help drive traffic to your blog.\n9. Write about the challenges of staying motivated and inspired as a blogger.\n10. Share your process for brainstorming and creating blog post ideas.", response_metadata={'token_usage': {'completion_tokens': 138, 'prompt_tokens': 16, 'total_tokens': 154}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-37b883f5-038c-463f-bc3f-0d6898c61faa-0')

In [21]:
print(r.content)

Please write a blog post about the impact of social media on mental health. Make sure to include statistics and case studies to support your argument. Additionally, discuss potential solutions to mitigate the negative effects of social media on mental well-being. Be sure to cite your sources and use proper formatting for your post.


In [18]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

target = ('a ai anime filter website. You can upload a photo, '
'and then get a anime image with styles like Clay, 3D, video game, Emoji, pixels, toy, and so on.'
)

idea_sys = (
        "As a creative ideas expert, "
        "your task is to generate several innovative approaches related to the topic."
        " Each approach should be explored from a unique angle to "
        "provide a multi-faceted perspective on the theme."
        "\n===\n{format_instructions}\n===\n topic: `{topic}`"
        )
# parser = JsonOutputParser(pydantic_object=IdeaGen)
parser = JsonOutputParser(pydantic_object=IdeaList)
idea_prompt = PromptTemplate(
    template=idea_sys, input_variables=["topic"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [19]:
print(f'{target=}')
chain = idea_prompt | llm | parser
chain.invoke({"topic": target})

target='a blog writer'


{'idea_list': [{'point': 'Utilize storytelling techniques to engage readers',
   'outline': 'Craft narratives that captivate the audience and evoke emotions to keep them coming back for more'},
  {'point': 'Experiment with different writing styles and tones',
   'outline': 'Explore a variety of tones from formal to conversational, and styles such as listicles, how-tos, and personal essays to cater to diverse reader preferences'},
  {'point': 'Collaborate with guest bloggers and experts',
   'outline': 'Bring fresh perspectives and expertise to the blog by inviting guest contributors to share their insights and knowledge with the audience'},
  {'point': 'Optimize SEO for increased visibility',
   'outline': 'Implement keyword research, meta tags, and backlinking strategies to improve search engine rankings and attract more organic traffic to the blog'}]}

In [ ]:
blog_prompt = """
You are a blog writer, are good at write articles according given {point} and {outline}.

"""
cn = """
Prompt:

角色： 博客作家 (Bó kè jiā zuò zhě)

任务： 撰写一篇博客文章 (Zhuān xiě yī piān bó kè wén zhāng)

目标： 清晰地解释一件事情 (Qīng xī de jiě shì yī jiàn shì qíng)

给定信息：

观点 (Guǎn diǎn)
大纲 (Dà gāng)
要求：

遵循博客文章写作的最佳实践 (Zūncún bó kè wén zhāng xiě zuò de zuì jiā shí xiàn)
使用清晰简洁的语言 (Shǐ yòng qīng xī jiǎn jié de yǔ yán)
举例说明 (Jǔ lì shuo míng)
使文章引人入胜 (Shǐ wén zhāng yǐn rén rù shèng)
示例：

您可以选择一个您熟悉的话题，并就该话题表达您的观点。
您可以使用大纲来组织您的想法，并确保您的文章内容连贯。
您可以使用示例来说明您的观点，并使您的文章更具吸引力。
提示：

在开始写作之前，请务必仔细阅读给定信息。
考虑您的目标受众，并确保您的文章适合他们。
使用各种写作技巧来使您的文章引人入胜。
校对您的文章以确保没有错误。
"""

In [28]:
idea_prompt = ChatPromptTemplate.from_messages([
    ("system", idea_sys),
    ("user", "{input}")
])
idea_chain = idea_prompt | llm

In [27]:
blog_writer_prompt = ChatPromptTemplate.from_messages(
    [("system", """
Please write a blog post about the impact of social media on mental health.
Make sure to include statistics and case studies to support your argument.
Additionally, discuss potential solutions to mitigate the negative effects of social media on mental well-being.
Be sure to cite your sources and use proper formatting for your post.
"""),
("user", "{input}")
     ])

In [ ]:
chain = blog_writer_prompt | llm
chain.invoke({"input": "write a blog"})

In [23]:
cn2en_prompt = """
请将以下中文段落翻译成英文。在翻译过程中，请遵循以下步骤以确保翻译的准确性和流畅性：
理解原文：阅读中文段落，确保完全理解其含义。
词汇对应：识别并确定每个中文单词或短语的准确英文对应词汇。{term_option}
语法结构：保持英文语法的正确性，注意时态、语态和主谓一致。
流畅性检查：确保翻译后的英文读起来自然流畅，符合英语表达习惯。
文化适应性：适当调整翻译，以传达原文的文化内涵和语境。
校对：完成翻译后，仔细校对以避免拼写和语法错误。
中文文章如下：```
{article}
```
"""
pt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
pt = PromptTemplate.from_template()


NameError: name 'ChatPromptTemplate' is not defined

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

In [5]:
llm.invoke("how can langsmith help with testing?")


AIMessage(content='Langsmith can help with testing in the following ways:\n\n1. Automated testing: Langsmith can be used to write automated test scripts for software applications, enabling developers to quickly and efficiently test their code.\n\n2. Test data generation: Langsmith can generate test data for a wide range of scenarios, helping developers to ensure that their software is robust and can handle various inputs.\n\n3. Performance testing: Langsmith can be used to simulate high loads on a system, allowing developers to test the performance of their applications under stress.\n\n4. Integration testing: Langsmith can be used to test the integration of different components of a software application, ensuring that they work together seamlessly.\n\n5. Regression testing: Langsmith can help automate regression testing, allowing developers to quickly identify and fix any issues that arise when changes are made to the codebase.\n\nOverall, Langsmith can help streamline the testing pro

In [ ]:
en2cn_prompt = """

请将以下英文段落翻译成中文。在翻译过程中，请遵循以下步骤以确保翻译的准确性和自然性：

理解原文：仔细阅读英文段落，确保完全理解其含义和上下文。
词汇翻译：将英文单词或短语逐一翻译成中文，注意选择最合适的中文词汇。
语法调整：根据中文的语法习惯调整句子结构，确保翻译后的中文句子语法正确。
保持原意：在翻译过程中，确保保留原文的意思和语气。
文化适应：适当调整翻译，使其更符合中文读者的文化背景和阅读习惯。
流畅性优化：确保翻译后的中文读起来流畅自然，无生硬的直译表达。

"""

cn2en_prompt = """
请将以下中文段落翻译成英文。在翻译过程中，请遵循以下步骤以确保翻译的准确性和流畅性：
理解原文：阅读中文段落，确保完全理解其含义。
词汇对应：识别并确定每个中文单词或短语的准确英文对应词汇。{term_option}
语法结构：保持英文语法的正确性，注意时态、语态和主谓一致。
流畅性检查：确保翻译后的英文读起来自然流畅，符合英语表达习惯。
文化适应性：适当调整翻译，以传达原文的文化内涵和语境。
校对：完成翻译后，仔细校对以避免拼写和语法错误。
中文文章如下：```
{article}
```
"""
term_option_find = "对专业词汇需要找到对应的中文词，如果没有则保留原词"
term_option_keep = "对专业词汇需要要保留原词"
cn2en_prompt_en = """
Please translate the following Chinese paragraph into English. During the translation process, please follow the steps below to ensure the accuracy and fluency of the translation:

Understand the Original Text: Read the Chinese paragraph and make sure you fully understand its meaning.
Lexical Correspondence: Identify and determine the accurate English corresponding words or phrases for each Chinese word or phrase. {term_option}
Grammar Structure: Maintain the correctness of English grammar, paying attention to tense, voice, and subject-verb agreement.
Fluency Check: Ensure that the translated English reads naturally and fluently, conforming to English expression habits.
Cultural Adaptability: Appropriately adjust the translation to convey the original text's cultural content and context.
Proofreading: After completing the translation, carefully proofread to avoid any spelling or grammatical errors.
There is the paragraph:
```{article}```
"""
term = """When encountering professional terminology, one should attempt to find the corresponding Chinese term. If no equivalent is available, the original term should be retained."""
term_option_keep = "When encountering professional terminology, the original term should be retained"


cn_article = """ """

en_article = """ """



# pip install


In [4]:
from google.colab import userdata
ok = userdata.get('openai_first')

In [5]:
!pip install langchain-openai
!pip install openai tiktoken backoff
!pip install llama-index langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━